In [40]:
import pandas as pd, numpy as np

#Reading data
df_clinicalsample = pd.read_csv("C:/Users/koush/Desktop/ML-Bio-HW2/data/brca_tcga_pub2015/data_clinical_sample.txt", sep="\t", skiprows=4)
df_clinicalsample.replace('[Not Available]', np.nan, inplace=True)
df_mutations = pd.read_csv("C:/Users/koush/Desktop/ML-Bio-HW2/data/brca_tcga_pub2015/data_mutations.txt", sep="\t")
df_mutations['PATIENT_ID'] = df_mutations['Tumor_Sample_Barcode'].apply(lambda x: x[:-3])
df = pd.merge(df_mutations, df_clinicalsample, on='PATIENT_ID', how='left')
df.head()
#getting only the rows we need
cancer = df[['Hugo_Symbol','PATIENT_ID','Variant_Classification',
             'Variant_Type','Transcript_ID','Feature','Gene',
             'CANCER_TYPE','CANCER_TYPE_DETAILED',]]
cancer = cancer.loc[(cancer['CANCER_TYPE_DETAILED']=='Breast Invasive Lobular Carcinoma') | (cancer['CANCER_TYPE_DETAILED']=='Breast Invasive Ductal Carcinoma')]
#Getting the target variables
df_target = cancer[['PATIENT_ID','CANCER_TYPE_DETAILED']].drop_duplicates()
print(cancer.shape)
display(cancer.head())

(56933, 9)


,Hugo_Symbol,PATIENT_ID,Variant_Classification,Variant_Type,Transcript_ID,Feature,Gene,CANCER_TYPE,CANCER_TYPE_DETAILED
0,PTGER3,TCGA-B6-A0IG,Missense_Mutation,SNP,ENST00000356595,ENST00000356595,ENSG00000050628,Invasive Breast Carcinoma,Breast Invasive Ductal Carcinoma
1,FLG,TCGA-B6-A0IG,Missense_Mutation,SNP,ENST00000368799,ENST00000368799,ENSG00000143631,Invasive Breast Carcinoma,Breast Invasive Ductal Carcinoma
2,GPR52,TCGA-B6-A0IG,Silent,SNP,ENST00000367685,ENST00000367685,ENSG00000203737,Invasive Breast Carcinoma,Breast Invasive Ductal Carcinoma
3,SLC35F3,TCGA-B6-A0IG,Silent,SNP,ENST00000366618,ENST00000366618,ENSG00000183780,Invasive Breast Carcinoma,Breast Invasive Ductal Carcinoma
4,OR2T3,TCGA-B6-A0IG,Missense_Mutation,SNP,ENST00000359594,ENST00000359594,ENSG00000196539,Invasive Breast Carcinoma,Breast Invasive Ductal Carcinoma


In [41]:
col = ["PATIENT_ID"]+cancer['Hugo_Symbol'].drop_duplicates().sort_values().values.tolist() 

In [42]:
dataf = pd.DataFrame(columns=col)

In [43]:
dataf["PATIENT_ID"] = cancer['PATIENT_ID'].drop_duplicates().values

In [44]:
for i in dataf.iterrows():
    pat = i[1]['PATIENT_ID']
    hug = cancer[cancer['PATIENT_ID']==pat]['Hugo_Symbol'].values
    val,count = np.unique(hug,return_counts=True)
    dataf.loc[i[0],val]=count

In [45]:
dataf.fillna(0,inplace=True)

In [46]:
dataf = dataf.merge(df_target,how="inner",on="PATIENT_ID")

In [47]:
dataf.head()

,PATIENT_ID,A1CF,A2M,A2ML1,A4GALT,A4GNT,AAAS,AACS,AADAC,AADACL2,...,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3,CANCER_TYPE_DETAILED
0,TCGA-B6-A0IG,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Ductal Carcinoma
1,TCGA-BH-A0HQ,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Ductal Carcinoma
2,TCGA-BH-A18G,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Ductal Carcinoma
3,TCGA-A1-A0SD,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Ductal Carcinoma
4,TCGA-A1-A0SF,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Breast Invasive Ductal Carcinoma


Linear Discriminant Analysis with KFold cross validation

In [48]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score

clf = LinearDiscriminantAnalysis()
scores_clf = cross_val_score(clf, dataf.iloc[:,1:-1], dataf['CANCER_TYPE_DETAILED'], cv=5,verbose=3,scoring="f1_weighted")

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END ................................ score: (test=0.719) total time=   5.9s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.9s remaining:    0.0s


[CV] END ................................ score: (test=0.755) total time=   5.5s


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   11.5s remaining:    0.0s


[CV] END ................................ score: (test=0.716) total time=   5.3s
[CV] END ................................ score: (test=0.718) total time=   5.1s
[CV] END ................................ score: (test=0.725) total time=   5.5s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   27.7s finished


In [49]:
scores_clf

array([0.71904266, 0.75492832, 0.71624098, 0.71790707, 0.72546478])

Naive Bayes with KFold cross validation

In [50]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
scores_clf1 = cross_val_score(clf, dataf.iloc[:,1:-1], dataf['CANCER_TYPE_DETAILED'], cv=5,verbose=3,scoring="f1_weighted")

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END ................................ score: (test=0.698) total time=   1.2s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s


[CV] END ................................ score: (test=0.698) total time=   0.7s


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.1s remaining:    0.0s


[CV] END ................................ score: (test=0.707) total time=   0.6s
[CV] END ................................ score: (test=0.707) total time=   0.6s
[CV] END ................................ score: (test=0.707) total time=   0.6s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    4.3s finished


In [51]:
scores_clf1

array([0.69776228, 0.69776228, 0.70661811, 0.70661811, 0.70661811])

Logistic Regression with KFold cross validation

In [52]:
from sklearn.linear_model import LogisticRegression


clf = LogisticRegression()
scores_clf2 = cross_val_score(clf, dataf.iloc[:,1:-1], dataf['CANCER_TYPE_DETAILED'], cv=5,verbose=3,scoring="f1_weighted")

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END ................................ score: (test=0.894) total time=   1.3s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s


[CV] END ................................ score: (test=0.848) total time=   0.8s


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.2s remaining:    0.0s


[CV] END ................................ score: (test=0.882) total time=   0.9s
[CV] END ................................ score: (test=0.837) total time=   0.9s
[CV] END ................................ score: (test=0.890) total time=   0.9s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    5.2s finished


In [53]:
scores_clf2

array([0.89405396, 0.84815486, 0.88221201, 0.83708462, 0.88984892])